In [38]:
import pandas as pd
import numpy as np


#Load the Dataset

esg = pd.read_csv("/files/capstone_project/esg.csv", sep=";", engine="python")
beta = pd.read_csv("/files/capstone_project/beta.csv",  sep=";", engine="python")
debt_equity = pd.read_csv("/files/capstone_project/debt_equity.csv",  sep=";", engine="python")
pe = pd.read_csv("/files/capstone_project/pe.csv",  sep=";", engine="python")
returns = pd.read_csv("/files/capstone_project/totreturn.csv",  sep=";", engine="python")
roe = pd.read_csv("/files/capstone_project/roe.csv",  sep=";", engine="python")
main = pd.read_csv("/files/capstone_project/Main.csv", sep=";", engine="python")



In [4]:
sector = pd.read_csv("/files/capstone_project/sectors.csv", sep=",", engine="python")

In [14]:
marketcap = pd.read_csv("/files/capstone_project/marketcap.csv", sep=";", engine="python")

In [5]:
sector.columns

Index(['Symbol', 'Name', 'Address', 'Sector', 'Industry',
       'Full Time Employees', 'Description', 'Total ESG Risk score',
       'Environment Risk Score', 'Governance Risk Score', 'Social Risk Score',
       'Controversy Level', 'Controversy Score', 'ESG Risk Percentile',
       'ESG Risk Level'],
      dtype='object')

In [6]:
sector.head()

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
0,ENPH,"Enphase Energy, Inc.","47281 Bayside Parkway\nFremont, CA 94538\nUnit...",Technology,Solar,"3,157","Enphase Energy, Inc., together with its subsid...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EMN,Eastman Chemical Company,"200 South Wilcox Drive\nKingsport, TN 37662\nU...",Basic Materials,Specialty Chemicals,"14,000",Eastman Chemical Company operates as a special...,25.3,12.8,6.6,5.8,Moderate Controversy Level,2.0,50th percentile,Medium
2,DPZ,Domino's Pizza Inc.,"30 Frank Lloyd Wright Drive\nAnn Arbor, MI 481...",Consumer Cyclical,Restaurants,"6,500","Domino's Pizza, Inc., through its subsidiaries...",29.2,10.6,6.3,12.2,Moderate Controversy Level,2.0,66th percentile,Medium
3,DAY,"Dayforce, Inc.","3311 East Old Shakopee Road\nMinneapolis, MN 5...",Technology,Software - Application,"9,084","Dayforce Inc., together with its subsidiaries,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DVA,Davita Inc.,"2000 16th Street\nDenver, CO 80202\nUnited States",Healthcare,Medical Care Facilities,"70,000",DaVita Inc. provides kidney dialysis services ...,22.6,0.1,8.4,14.1,Moderate Controversy Level,2.0,38th percentile,Medium


In [7]:
#I had a problem with merging because of different names, it gave me lot ot NaN. I asked ChatGPT to resolve the issue by creating a new column with cleaned names
import re

def clean_name(x):
    if pd.isna(x):
        return x
    x = x.lower().strip()

    # enlever les virgules, points, parenthèses, etc.
    x = re.sub(r"[.,]", "", x)

    # enlever les suffixes (inc, inc, corp, corporation, ltd, llc...)
    x = re.sub(r"\b(inc|incorporated|corp|corporation|ltd|llc)\b", "", x)

    # enlever espaces multiples
    x = " ".join(x.split())

    return x

sector["Name_clean"] = sector["Name"].apply(clean_name)
main["Name_clean"] = main["Name"].apply(clean_name)



KeyError: 'Name'

In [108]:
main.head()

,Ticker,Name,Name_clean
2,CCI.N,Crown Castle Inc,crown castle
3,CRWD.OQ,CrowdStrike Holdings Inc,crowdstrike holdings
4,BLK.N,BlackRock Inc,blackrock
5,GPN.N,Global Payments Inc,global payments
6,UNP.N,Union Pacific Corp,union pacific


In [107]:
sector.head()



,Name,Sector,Industry,Name_clean
0,"Enphase Energy, Inc.",Technology,Solar,enphase energy
1,Eastman Chemical Company,Basic Materials,Specialty Chemicals,eastman chemical company
2,Domino's Pizza Inc.,Consumer Cyclical,Restaurants,domino's pizza
3,"Dayforce, Inc.",Technology,Software - Application,dayforce
4,Davita Inc.,Healthcare,Medical Care Facilities,davita


In [109]:
#Merge first my main dataset with the sector dataset to have the sector information in my main dataset
merge = main.merge(sector, on = "Name_clean", how = "left")


In [9]:
merge.head()

,Ticker,Name_x,Name_clean,Name_y,Sector,Industry
0,CCI.N,Crown Castle Inc,crown castle,Crown Castle Inc.,Real Estate,REIT - Specialty
1,CRWD.OQ,CrowdStrike Holdings Inc,crowdstrike holdings,NaN,NaN,NaN
2,BLK.N,BlackRock Inc,blackrock,"Blackrock, Inc.",Financial Services,Asset Management
3,GPN.N,Global Payments Inc,global payments,"Global Payments, Inc.",Industrials,Specialty Business Services
4,UNP.N,Union Pacific Corp,union pacific,Union Pacific Corp.,Industrials,Railroads


In [10]:
#Due to missing Variables, I decided to fill manually the missing sectors and industries of the companies with Yahoo Finance data


merge.loc[merge["Name_clean"] == "crowdstrike holdings", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "sherwin-williams co", ["Sector", "Industry"]] = ["Materials", "Specialty Chemicals"]
merge.loc[merge["Name_clean"] == "general electric co", ["Sector", "Industry"]] = ["Industrials", "Industrial Conglomerates"]
merge.loc[merge["Name_clean"] == "datadog", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "fox", ["Sector", "Industry"]] = ["Communication Services", "Broadcasting"]
merge.loc[merge["Name_clean"] == "workday", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "charter communications", ["Sector", "Industry"]] = ["Communication Services", "Telecom Services"]
merge.loc[merge["Name_clean"] == "cf industries holdings", ["Sector", "Industry"]] = ["Materials", "Agricultural Chemicals"]
merge.loc[merge["Name_clean"] == "procter & gamble co", ["Sector", "Industry"]] = ["Consumer Staples", "Household Products"]
merge.loc[merge["Name_clean"] == "eaton plc", ["Sector", "Industry"]] = ["Industrials", "Electrical Equipment"]
merge.loc[merge["Name_clean"] == "molson coors beverage co", ["Sector", "Industry"]] = ["Consumer Staples", "Beverages"]


In [12]:
merge.loc[merge["Name_clean"] == "block", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "godaddy", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "interactive brokers group", ["Sector", "Industry"]] = ["Financials", "Investment Banking"]
merge.loc[merge["Name_clean"] == "ford motor co", ["Sector", "Industry"]] = ["Consumer Discretionary", "Automobiles"]
merge.loc[merge["Name_clean"] == "cigna group", ["Sector", "Industry"]] = ["Healthcare", "Managed Health Care"]
merge.loc[merge["Name_clean"] == "aon plc", ["Sector", "Industry"]] = ["Financials", "Insurance Brokers"]
merge.loc[merge["Name_clean"] == "clorox co", ["Sector", "Industry"]] = ["Consumer Staples", "Household Products"]
merge.loc[merge["Name_clean"] == "trade desk", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "colgate-palmolive co", ["Sector", "Industry"]] = ["Consumer Staples", "Personal Products"]
merge.loc[merge["Name_clean"] == "united parcel service", ["Sector", "Industry"]] = ["Industrials", "Air Freight & Logistics"]
merge.loc[merge["Name_clean"] == "mccormick & company", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"]
merge.loc[merge["Name_clean"] == "cooper companies", ["Sector", "Industry"]] = ["Healthcare", "Medical Devices"]


In [13]:
merge.loc[merge["Name_clean"] == "walt disney co", ["Sector", "Industry"]] = ["Communication Services", "Entertainment"]
merge.loc[merge["Name_clean"] == "doordash", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "lennar", ["Sector", "Industry"]] = ["Consumer Discretionary", "Homebuilding"]
merge.loc[merge["Name_clean"] == "paramount skydance", ["Sector", "Industry"]] = ["Communication Services", "Entertainment"]
merge.loc[merge["Name_clean"] == "mondelez international", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"]
merge.loc[merge["Name_clean"] == "apa (us)", ["Sector", "Industry"]] = ["Energy", "Oil & Gas Exploration"]
merge.loc[merge["Name_clean"] == "general motors co", ["Sector", "Industry"]] = ["Consumer Discretionary", "Automobiles"]
merge.loc[merge["Name_clean"] == "airbnb", ["Sector", "Industry"]] = ["Consumer Discretionary", "Travel Services"]
merge.loc[merge["Name_clean"] == "hartford insurance group", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "applovin", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "nextera energy", ["Sector", "Industry"]] = ["Utilities", "Electric Utilities"]
merge.loc[merge["Name_clean"] == "erie indemnity co", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "hewlett packard enterprise co", ["Sector", "Industry"]] = ["Technology", "Information Technology Services"]
merge.loc[merge["Name_clean"] == "berkshire hathaway", ["Sector", "Industry"]] = ["Financials", "Diversified Financial Services"]


In [14]:
merge.loc[merge["Name_clean"] == "norwegian cruise line holdings", ["Sector", "Industry"]] = ["Consumer Discretionary", "Hotels, Resorts & Cruise Lines"]
merge.loc[merge["Name_clean"] == "kroger co", ["Sector", "Industry"]] = ["Consumer Staples", "Food Retail"]
merge.loc[merge["Name_clean"] == "archer-daniels-midland co", ["Sector", "Industry"]] = ["Consumer Staples", "Agricultural Products"]
merge.loc[merge["Name_clean"] == "universal health services", ["Sector", "Industry"]] = ["Healthcare", "Healthcare Services"]
merge.loc[merge["Name_clean"] == "westinghouse air brake technology", ["Sector", "Industry"]] = ["Industrials", "Railroads"]
merge.loc[merge["Name_clean"] == "bxp", ["Sector", "Industry"]] = ["Real Estate", "REITs"]
merge.loc[merge["Name_clean"] == "best buy co", ["Sector", "Industry"]] = ["Consumer Discretionary", "Specialty Retail"]
merge.loc[merge["Name_clean"] == "willis towers watson plc", ["Sector", "Industry"]] = ["Financials", "Insurance Brokers"]
merge.loc[merge["Name_clean"] == "news", ["Sector", "Industry"]] = ["Communication Services", "Publishing"]
merge.loc[merge["Name_clean"] == "jack henry & associates", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "allstate corp", ["Sector", "Industry"]] = ["Financials", "Insurance"]


In [15]:
merge.loc[merge["Name_clean"] == "alphabet inc", ["Sector", "Industry"]] = ["Communication Services", "Internet Content & Information"]
merge.loc[merge["Name_clean"] == "expand energy", ["Sector", "Industry"]] = ["Energy", "Oil & Gas Exploration"]
merge.loc[merge["Name_clean"] == "weyerhaeuser co", ["Sector", "Industry"]] = ["Materials", "Forest Products"]
merge.loc[merge["Name_clean"] == "lennox international", ["Sector", "Industry"]] = ["Industrials", "Building Products"]
merge.loc[merge["Name_clean"] == "dell technologies inc", ["Sector", "Industry"]] = ["Technology", "Information Technology Services"]
merge.loc[merge["Name_clean"] == "mosaic co", ["Sector", "Industry"]] = ["Materials", "Agricultural Chemicals"]
merge.loc[merge["Name_clean"] == "j b hunt transport services", ["Sector", "Industry"]] = ["Industrials", "Trucking"]
merge.loc[merge["Name_clean"] == "vulcan materials co", ["Sector", "Industry"]] = ["Materials", "Construction Materials"]
merge.loc[merge["Name_clean"] == "hershey co", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"] 
merge.loc[merge["Name_clean"] == "smurfit westrock plc", ["Sector", "Industry"]] = ["Materials", "Paper Products"]
merge.loc[merge["Name_clean"] == "incyte", ["Sector", "Industry"]] = ["Healthcare", "Biotechnology"]


In [16]:
merge.loc[merge["Name_clean"] == "southern co", ["Sector", "Industry"]] = ["Utilities", "Electric Utilities"]
merge.loc[merge["Name_clean"] == "interpublic group of companies", ["Sector", "Industry"]] = ["Communication Services", "Advertising"]
merge.loc[merge["Name_clean"] == "allegion plc", ["Sector", "Industry"]] = ["Industrials", "Building Products"]
merge.loc[merge["Name_clean"] == "baker hughes co", ["Sector", "Industry"]] = ["Energy", "Oil & Gas Equipment & Services"]
merge.loc[merge["Name_clean"] == "coca-cola co", ["Sector", "Industry"]] = ["Consumer Staples", "Beverages"]
merge.loc[merge["Name_clean"] == "deere & co", ["Sector", "Industry"]] = ["Industrials", "Farm & Heavy Construction Machinery"]
merge.loc[merge["Name_clean"] == "solstice advanced materials", ["Sector", "Industry"]] = ["Materials", "Specialty Chemicals"]
merge.loc[merge["Name_clean"] == "robinhood markets", ["Sector", "Industry"]] = ["Financials", "Investment Banking"]
merge.loc[merge["Name_clean"] == "a o smith", ["Sector", "Industry"]] = ["Industrials", "Building Products"]
merge.loc[merge["Name_clean"] == "boeing co", ["Sector", "Industry"]] = ["Industrials", "Aerospace & Defense"]
merge.loc[merge["Name_clean"] == "equinix", ["Sector", "Industry"]] = ["Real Estate", "REITs"]
merge.loc[merge["Name_clean"] == "royal caribbean cruises", ["Sector", "Industry"]] = ["Consumer Discretionary", "Hotels, Resorts & Cruise Lines"]
merge.loc[merge["Name_clean"] == "genuine parts co", ["Sector", "Industry"]] = ["Consumer Discretionary", "Auto Parts & Equipment"] 
merge.loc[merge["Name_clean"] == "seagate technology holdings plc", ["Sector", "Industry"]] = ["Technology", "Computer Hardware"]


In [17]:
merge.loc[merge["Name_clean"] == "3m co", ["Sector", "Industry"]] = ["Industrials", "Industrial Conglomerates"]
merge.loc[merge["Name_clean"] == "amcor plc", ["Sector", "Industry"]] = ["Materials", "Packaging"]
merge.loc[merge["Name_clean"] == "texas pacific land", ["Sector", "Industry"]] = ["Real Estate", "REITs"]
merge.loc[merge["Name_clean"] == "marriott international", ["Sector", "Industry"]] = ["Consumer Discretionary", "Hotels, Resorts & Cruise Lines"]
merge.loc[merge["Name_clean"] == "emcor group", ["Sector", "Industry"]] = ["Industry", "Engineering & Construction"]
merge.loc[merge["Name_clean"] == "chubb", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "warner bros discovery", ["Sector", "Industry"]] = ["Communication Services", "Entertainment"]
merge.loc[merge["Name_clean"] == "meta platforms", ["Sector", "Industry"]] = ["Communication Services", "Internet Content & Information"]
merge.loc[merge["Name_clean"] == "williams-sonoma", ["Sector", "Industry"]] = ["Consumer Discretionary", "Specialty Retail"]
merge.loc[merge["Name_clean"] == "coinbase global", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "brown-forman", ["Sector", "Industry"]] = ["Consumer Staples", "Beverages"]
merge.loc[merge["Name_clean"] == "te connectivity plc", ["Sector", "Industry"]] = ["Industrials", "Electrical Components & Equipment"]
merge.loc[merge["Name_clean"] == "estee lauder companies", ["Sector", "Industry"]] = ["Consumer Staples", "Personal Products"]


In [18]:
merge.loc[merge["Name_clean"] == "lyondellbasell industries nv", ["Sector", "Industry"]] = ["Materials", "Specialty Chemicals"]
merge.loc[merge["Name_clean"] == "j m smucker co", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"]
merge.loc[merge["Name_clean"] == "american water works co", ["Sector", "Industry"]] = ["Utilities", "Water Utilities"]
merge.loc[merge["Name_clean"] == "travelers companies", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "dte energy co", ["Sector", "Industry"]] = ["Utilities", "Electric Utilities"]
merge.loc[merge["Name_clean"] == "appollo global management", ["Sector", "Industry"]] = ["Financials", "Asset Management"]
merge.loc[merge["Name_clean"] == "tko group holdings", ["Sector", "Industry"]] = ["Communication Services", "Entertainment"]
merge.loc[merge["Name_clean"] == "qnity electronics", ["Sector", "Industry"]] = ["Technology", "Computer Hardware"]
merge.loc[merge["Name_clean"] == "campbell's co", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"]
merge.loc[merge["Name_clean"] == "palantir technologies", ["Sector", "Industry"]] = ["Technology", "Software"]
merge.loc[merge["Name_clean"] == "halliburton co", ["Sector", "Industry"]] = ["Energy", "Oil & Gas Equipment & Services"]
merge.loc[merge["Name_clean"] == "air products and chemicals", ["Sector", "Industry"]] = ["Materials", "Industrial Gases"]
merge.loc[merge["Name_clean"] == "slb nv", ["Sector", "Industry"]] = ["Energy", "Oil & Gas Equipment & Services"]
merge.loc[merge["Name_clean"] == "charles schwab", ["Sector", "Industry"]] = ["Financials", "Investment Banking"]
merge.loc[merge["Name_clean"] == "eli lilly and co", ["Sector", "Industry"]] = ["Healthcare", "Pharmaceuticals"]


In [22]:
merge.loc[merge["Name_clean"] == "kraft heinz co", ["Sector", "Industry"]] = ["Consumer Staples", "Food Products"]
merge.loc[merge["Name_clean"] == "american express co", ["Sector", "Industry"]] = ["Financials", "Consumer Finance"]
merge.loc[merge["Name_clean"] == "w r berkley", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "westinghouse air brake technologies", ["Sector", "Industry"]] = ["Industrials", "Railroads"]
merge.loc[merge["Name_clean"] == "allstate", ["Sector", "Industry"]] = ["Financials", "Insurance"]
merge.loc[merge["Name_clean"] == "alphabet", ["Sector", "Industry"]] = ["Communication Services", "Internet Content & Information"]
merge.loc[merge["Name_clean"] == "dell technologies", ["Sector", "Industry"]] = ["Technology", "Information Technology Services"]
merge.loc[merge["Name_clean"] == "kkr & co", ["Sector", "Industry"]] = ["Financials", "Asset Management"]
merge.loc[merge["Name_clean"] == "apollo global management", ["Sector", "Industry"]] = ["Financials", "Asset Management"]


In [ ]:
#Save the new merged dataset with all the sectors and industries filled
merge.to_csv("new_merge.csv", sep=";", index=False)

In [10]:
#Rename Columns for Merging

esg = esg.rename(columns={
    "ESG Score": "ESG_Score",
    "Company Ticker": "Ticker",
})

beta = beta.rename(columns={
    "Company Ticker": "Ticker",
    "Beta 5 Year": "Beta_Value",
    })
debt_equity = debt_equity.rename(columns={
    "Company Ticker": "Ticker",
})

pe = pe.rename(columns={
    "Company Ticker": "Ticker",
    "P/E Ratio": "PE_Ratio",
})

returns = returns.rename(columns={
    "Company Ticker": "Ticker",
    "Total Return": "Tot_Return",
})

roe = roe.rename(columns={ 
    "Company Ticker": "Ticker",
    "ROE ( % )": "ROE_Value",
})

main = main.rename(columns={
    "Identifier (RIC)": "Ticker",
    "Company_Name": "Name",
})



In [55]:
merge = merge.drop(columns=["Name_y"])


In [15]:
#Before Merging, I will create a "Year" column in each dataset to facilitate the merging process
list = ["esg", "beta", "debt_equity", "pe", "returns", "roe", "marketcap"]
for dfs in list:
    globals()[dfs]["Year"] = pd.to_datetime(globals()[dfs]["Date"]).dt.year

/tmp/ipykernel_685/3888461614.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  globals()[dfs]["Year"] = pd.to_datetime(globals()[dfs]["Date"]).dt.year
/tmp/ipykernel_685/3888461614.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  globals()[dfs]["Year"] = pd.to_datetime(globals()[dfs]["Date"]).dt.year
/tmp/ipykernel_685/3888461614.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  globals()[dfs]["Year"] = pd.to_datetime(globals()[dfs]["Date"]).dt.year
/tmp/ipykernel_685/3888461614.py:4: UserWarning: Could not infer format, so each element will be parsed individually, 

In [67]:
#First I'm going to merge all the different datasets with the main dataset that contains now the sector and industry information
data = merge.merge(esg, on= "Ticker", how="left") 

In [ ]:
#Now I will merge the other datasets one by one

data = data.merge(debt_equity, on=["Ticker", "Year"], how="left")  
data = data.merge(marketcap, on=["Ticker", "Year"], how="left")

,Ticker,Name_x,Name_clean,Sector,Industry,Date_x,ESG Global Score,Year,Date_y,Market Cap,Size,Date,Debt-to-Equity
0,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,31.12.14,"24,89243715",2014.0,NaN,NaN,NaN,31.12.14,"176,94015"
1,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,31.12.15,"30,65241605",2015.0,31.12.15,28854529490,"10,460214",31.12.15,"171,38638"
2,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,31.12.16,"33,82833598",2016.0,30.12.16,31289683876,"10,49540118",31.12.16,"161,0554"
3,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,31.12.17,"37,63093958",2017.0,29.12.17,45100597852,"10,6541823",31.12.17,"130,9629"
4,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,31.12.18,"61,04595033",2018.0,31.12.18,45065203514,"10,65384134",31.12.18,"144,17077"


In [82]:
#I tried to merge all at once, but it gave me errors because pandas create each time date_x and date_y, so I merged one by one 
data.drop(columns=["Date_x", "Date_y"], inplace=True)

In [76]:
data = data.merge(pe, on= ["Ticker","Year"], how="left")

In [80]:
data = data.merge(roe, on= ["Ticker", "Year"], how="left")


In [83]:
data = data.merge(beta, on= "Ticker", how="left")

In [ ]:
data = data.merge(returns, on=["Ticker", "Year"], how="left")

In [84]:
data.head()

,Ticker,Name_x,Name_clean,Sector,Industry,ESG Global Score,Year_x,Market Cap,Size,Debt-to-Equity,PE_Ratio,ROE_Value_x,ROE_Value_y,Date,Beta_Value,Year_y
0,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,"24,89243715",2014.0,NaN,NaN,"176,94015",NaN,"4,432040295","4,432040295",12.11.25,"0,956088055",2025.0
1,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,"30,65241605",2015.0,28854529490,"10,460214","171,38638","66,50920897","6,972680683","6,972680683",12.11.25,"0,956088055",2025.0
2,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,"33,82833598",2016.0,31289683876,"10,49540118","161,0554","66,76156036","4,42409853","4,42409853",12.11.25,"0,956088055",2025.0
3,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,"37,63093958",2017.0,45100597852,"10,6541823","130,9629","87,9865575","3,246854421","3,246854421",12.11.25,"0,956088055",2025.0
4,CCI.N,Crown Castle Inc,crown castle,Real Estate,REIT - Specialty,"61,04595033",2018.0,45065203514,"10,65384134","144,17077","81,88478992","4,257621215","4,257621215",12.11.25,"0,956088055",2025.0


In [4]:
#Now It's time to clean to have better dataset for analysis

data["Year"] = data["Year_x"].fillna(data["Year_y"])

data = data.drop(columns=["Year_x", "Year_y"])

In [5]:
data["ROE_Value"] = data["ROE_Value_x"].fillna(data["ROE_Value_y"])

data = data.drop(columns=["ROE_Value_x", "ROE_Value_y"])

In [6]:
data = data.drop(columns=["Name_x"])
data = data.rename(columns={"Name_clean": "Name"})

In [34]:
data = data.drop(columns=["Date"])

In [28]:
data = data[(data["Year"] >= 2015) & (data["Year"] <= 2024)].reset_index(drop=True)

In [24]:
#Before starting the analysis, I want to eplore more the dataset
data["Sector"].unique()


array(['Real Estate', 'Technology', 'Financial Services', 'Industrials',
       'Energy', 'Healthcare', 'Communication Services', 'Materials',
       'Consumer Defensive', 'Utilities', 'Consumer Cyclical',
       'Consumer Staples', 'Basic Materials', 'Financials',
       'Consumer Discretionary', 'Industry'], dtype=object)

In [26]:
data["Sector"].value_counts().sort_values(ascending=False)

Sector
Technology                841
Industrials               763
Healthcare                594
Financial Services        483
Consumer Cyclical         418
Real Estate               280
Utilities                 280
Consumer Defensive        220
Communication Services    204
Energy                    197
Consumer Staples          152
Financials                130
Basic Materials           113
Consumer Discretionary     89
Materials                  86
Industry                    9
Name: count, dtype: int64

In [33]:
data["Year"].value_counts().sort_values(ascending = True)

Year
2024.0    147
2023.0    226
2015.0    462
2020.0    484
2019.0    487
2018.0    487
2017.0    526
2021.0    535
2022.0    547
2016.0    610
Name: count, dtype: int64

In [36]:
data.columns

Index(['Ticker', 'Name', 'Sector', 'Industry', 'ESG Global Score',
       'Market Cap', 'Size', 'Debt-to-Equity', 'PE_Ratio', 'Beta_Value',
       'Year', 'ROE_Value', 'Tot_Return'],
      dtype='object')

In [39]:
data["Debt-to-Equity"] = data["Debt-to-Equity"].replace(0, np.nan)

In [40]:
#Create a variable for the next year return to predict for later

data["Next_Year_Return"] = data.groupby("Ticker")["Tot_Return"].shift(-1)

#Crate a Momentum variable to capture the effect of past returns on future returns

data["Momentum_1y"] = data.groupby("Ticker")["Tot_Return"].shift(1) 

In [54]:
#I want to create an interactive variable between ESG Score and Sector because some sectors matter more than others

high_impact = ["Energy", "Utilities", "Materials", "Industrials", "Consumer Discretionary", "Basic Materials", "Real Estate", "Consumer Cyclical"]

data["High_Impact_ESG"] = data["Sector"].apply(lambda x: 1 if x in high_impact else 0) 

data["ESG_x_Sector"] = data["ESG Global Score"] * data["High_Impact_ESG"]

In [ ]:
#Export the final dataset to save it for the analysis phase
data.to_csv("principal_dataset.csv", sep=";", index=False)